In [3]:
import openmeteo_requests

# Set up the client
om = openmeteo_requests.Client()

latitude = 40.7128
longitude = -74.0060

params = {
    "latitude": latitude,
    "longitude": longitude,
    "daily": [
        "temperature_2m_max",
        "temperature_2m_min",
        "precipitation_sum",
        "precipitation_probability_max",
        "wind_speed_10m_max",
        "wind_gusts_10m_max",
        "wind_direction_10m_dominant",
        "et0_fao_evapotranspiration"
    ],
    "timezone": "auto"
}

responses = om.weather_api("https://api.open-meteo.com/v1/forecast", params=params)
response = responses[0]

# Extract daily data
daily = response.Daily()
import pandas as pd

# Build a DataFrame
dates = pd.date_range(
    start=pd.to_datetime(daily.Time(), unit="s"),
    end=pd.to_datetime(daily.TimeEnd(), unit="s"),
    freq=pd.Timedelta(seconds=daily.Interval()),
    inclusive="left"
)
data = {
    "date": dates,
    "temperature_2m_max": daily.Variables(0).ValuesAsNumpy(),
    "temperature_2m_min": daily.Variables(1).ValuesAsNumpy(),
    "precipitation_sum": daily.Variables(2).ValuesAsNumpy(),
    "precipitation_probability_max": daily.Variables(3).ValuesAsNumpy(),
    "wind_speed_10m_max": daily.Variables(4).ValuesAsNumpy(),
    "wind_gusts_10m_max": daily.Variables(5).ValuesAsNumpy(),
    "wind_direction_10m_dominant": daily.Variables(6).ValuesAsNumpy(),
    "et0_fao_evapotranspiration": daily.Variables(7).ValuesAsNumpy(),
}
df = pd.DataFrame(data)

In [4]:
df

,date,temperature_2m_max,temperature_2m_min,precipitation_sum,precipitation_probability_max,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,et0_fao_evapotranspiration
0,2025-06-09 04:00:00,18.281000,16.181000,0.1,17.0,15.716793,33.119999,60.209934,1.051290
1,2025-06-10 04:00:00,22.681000,16.431000,23.5,53.0,15.629971,27.719999,135.000107,2.387123
2,2025-06-11 04:00:00,27.326500,15.481000,0.0,0.0,21.674870,42.119999,243.970444,6.292212
3,2025-06-12 04:00:00,30.226500,20.126499,3.4,11.0,17.877180,46.439999,278.788818,5.674593
4,2025-06-13 04:00:00,25.726500,19.226500,0.2,18.0,20.188908,21.599998,102.269852,5.869043
5,2025-06-14 04:00:00,22.626499,17.626499,0.0,41.0,20.063339,25.919998,84.029701,3.726940
6,2025-06-15 04:00:00,19.826500,16.126499,0.3,41.0,17.377226,22.680000,123.111343,2.444711


In [5]:
print(df)

                 date  temperature_2m_max  temperature_2m_min  \
0 2025-06-09 04:00:00           18.281000           16.181000   
1 2025-06-10 04:00:00           22.681000           16.431000   
2 2025-06-11 04:00:00           27.326500           15.481000   
3 2025-06-12 04:00:00           30.226500           20.126499   
4 2025-06-13 04:00:00           25.726500           19.226500   
5 2025-06-14 04:00:00           22.626499           17.626499   
6 2025-06-15 04:00:00           19.826500           16.126499   

   precipitation_sum  precipitation_probability_max  wind_speed_10m_max  \
0                0.1                           17.0           15.716793   
1               23.5                           53.0           15.629971   
2                0.0                            0.0           21.674870   
3                3.4                           11.0           17.877180   
4                0.2                           18.0           20.188908   
5                0.0         

In [5]:
import openmeteo_requests
import datetime
import pandas as pd

latitude = 40.7128
longitude = -74.0060

def _get_dates(daily):
    import pandas as pd

    dates = pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s"),
        end=pd.to_datetime(daily.TimeEnd(), unit="s"),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left",
    )
    return dates

def _extract_weather_data(daily, i):
    return {
        "temperature_max_c": float(daily.Variables(0).ValuesAsNumpy()[i]),
        "temperature_max_f": float(daily.Variables(0).ValuesAsNumpy()[i] * 9 / 5 + 32),
        "temperature_min_c": float(daily.Variables(1).ValuesAsNumpy()[i]),
        "temperature_min_f": float(daily.Variables(1).ValuesAsNumpy()[i] * 9 / 5 + 32),
        "precipitation_mm": float(daily.Variables(2).ValuesAsNumpy()[i]),
        "precipitation_in": float(daily.Variables(2).ValuesAsNumpy()[i] / 25.4),
        "precipitation_probability_max": float(daily.Variables(3).ValuesAsNumpy()[i]),
        "wind_speed_max_ms": float(daily.Variables(4).ValuesAsNumpy()[i]),
        "wind_speed_max_mph": float(daily.Variables(4).ValuesAsNumpy()[i] * 2.23694),
        "wind_gusts_max_ms": float(daily.Variables(5).ValuesAsNumpy()[i]),
        "wind_gusts_max_mph": float(daily.Variables(5).ValuesAsNumpy()[i] * 2.23694),
        "wind_direction_dominant_deg": float(daily.Variables(6).ValuesAsNumpy()[i]),
        "et0_evapotranspiration_mm": float(daily.Variables(7).ValuesAsNumpy()[i]),
        "et0_evapotranspiration_in": float(
            daily.Variables(7).ValuesAsNumpy()[i] / 25.4
        ),
    }

om = openmeteo_requests.Client()
# Fetch only yesterday's historical data
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)
params_hist = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": yesterday.isoformat(),
    "end_date": yesterday.isoformat(),
    "daily": [
        "temperature_2m_max",
        "temperature_2m_min",
        "precipitation_sum",
        "precipitation_probability_max",
        "wind_speed_10m_max",
        "wind_gusts_10m_max",
        "wind_direction_10m_dominant",
        "et0_fao_evapotranspiration",
    ],
    "timezone": "auto",
}
responses_hist = om.weather_api(
    "https://api.open-meteo.com/v1/forecast", params=params_hist
)
response_hist = responses_hist[0]
daily_hist = response_hist.Daily()
for i, date in enumerate(_get_dates(daily_hist)):
    data = _extract_weather_data(daily_hist, i)
    print(date)
    print(data)


2025-06-18 04:00:00
{'temperature_max_c': 29.380998611450195, 'temperature_max_f': 84.88579559326172, 'temperature_min_c': 18.0310001373291, 'temperature_min_f': 64.45580291748047, 'precipitation_mm': 0.20000000298023224, 'precipitation_in': 0.007874015718698502, 'precipitation_probability_max': 44.0, 'wind_speed_max_ms': 12.101570129394531, 'wind_speed_max_mph': 27.070486068725586, 'wind_gusts_max_ms': 17.639999389648438, 'wind_gusts_max_mph': 39.459617614746094, 'wind_direction_dominant_deg': 152.008056640625, 'et0_evapotranspiration_mm': 3.4888463020324707, 'et0_evapotranspiration_in': 0.13735616207122803}


In [2]:
daily_hist

In [3]:
daily_hist.Variables(0).ValuesAsNumpy()

array([29.380999], dtype=float32)